In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import requests
import re
import mechanicalsoup
import pandas as pd
import webbrowser
from selenium import webdriver
import time
import random
import mechanicalsoup
import json

### 네이버 시리즈 장르별 일간 top 100 크롤링

In [ ]:
naver_novel = pd.DataFrame()
label = ['title','genre','url','content','review_count']
genre_num = ['201','202','207','208','206','203','205','209']
for genre_id in genre_num:
    for i in range(0,6):
        url = f'https://series.naver.com/novel/top100List.series?rankingTypeCode=DAILY&categoryCode={genre_id}&page={i}'
        try:
            naver_headers = {}
            print(url, i)
            html = requests.get(url, headers = naver_headers, cookies = {}).text
            soup = BeautifulSoup(html, 'html.parser')
            product_count = len(soup.select('ul.comic_top_lst li'))
            for p in range(0,product_count):
                title = soup.select('ul.comic_top_lst li')[p].find_all('a')[1].get_text()
                print(title)
                detail_url = str("https://series.naver.com/")+soup.select('ul.comic_top_lst li')[p].find('a')['href']
                detail_html = requests.get(detail_url, headers=naver_headers, cookies={}).text
                #x = random.uniform(3, 5)
                #time.sleep(x)
                print('세부페이지: ', detail_url)
                body = BeautifulSoup(detail_html, 'html.parser')
                #            body = BeautifulSoup(driver.page_source, 'html.parser')
                browser = mechanicalsoup.StatefulBrowser()
                #더보기가 있으면 len = 2 없으면 len = 1
                #re.sub('\n','',url_soup.select('div._synopsis')[1].text)
                try :
                    a = body.select('div._synopsis')[1].text
                except:
                    a = body.select('div._synopsis')[0].text
                #desc에 쓰여있는 정규식 제거하기
                table = str.maketrans('\t\n\r\xa0\s+','       ')
                content = a.translate(table)
                #장르
                genre = body.select('li.info_lst li')[1].text
                #리뷰 관련
                bodyString = str(body)
                jsonStr = '{'+bodyString[bodyString.find("ghtProductInfo"):].split('}')[0].split('{')[1]+'}'
                jsonStr = jsonStr.replace("'",'"')
                dt = json.loads(jsonStr)
                origin_id = int(dt['originalProductId'])
                review_url = f'https://series.naver.com/comments/count.json?ticket=series5&objectId={origin_id}_0&lang=ko'
                review_html = requests.get(review_url, headers = naver_headers, cookies = {}).text
                review_count = json.loads(review_html, encoding="utf-8")['result']['comment']
                rx = [title,genre, detail_url, content,review_count]
                naver_novel = naver_novel.append(pd.Series(rx, index = label), ignore_index = True)
                x = random.uniform(0.5, 2.5)
                time.sleep(x)
        except Exception as e:
            print(e)
            continue
    #             if 'list index out of range' in str(e):
    #                 continue
    #             else:
    #                 time.sleep(3600)

### 네이버 웹툰 1570페이지까지 크롤링

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import requests
import re
import mechanicalsoup
import pandas as pd
import webbrowser
from selenium import webdriver
import time
import random
import json
import csv
# title, 마지막 연재일, 세부 url 가져오기
# page는 for문/ code = order
# 200page까지 돌리기
# label = ['title','genre','detail_url','date','review','content']
# f = open('naver_novel_tot.csv', 'a+', encoding = 'utf-8', newline='') # make new file if file doesn't exist

# wr = csv.writer(f)
# wr.writerow([label[0], label[1], label[2], label[3], label[4], label[5]])
f.close()
naver_novel = pd.DataFrame()
options = webdriver.ChromeOptions()
#agent가 없으면 bot으로 생각함
#options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko")
#driver = webdriver.Chrome(executable_path = './chromedriver',options=options)
#650~850
for page in range(640,650):
    print("페이지번호: ", page)
    # 페이지별 화면 켜기
    list_url = f'https://series.naver.com/novel/categoryProductList.series?categoryTypeCode=all&genreCode=&orderTypeCode=new&is&isFinished=false&page={page}'
    naver_headers = {}
    html = requests.get(list_url, headers=naver_headers, cookies={}).text
    soup = BeautifulSoup(html, 'html.parser')
    browser = mechanicalsoup.StatefulBrowser()
    product_count = len(soup.select('ul.lst_list div'))
    for i in range(0, product_count):
        try:
            rx = []
            k = soup.select('ul.lst_list div')[i]
            title = k.find('a')['title']
            #print('제목: ', title)
            #날짜는 첫번째에 들어온다!
            tt = k.select('p.info')[0]
            date = tt.text.replace('\n','').split('|')[2]
            detail_url = str("https://series.naver.com/") + k.find('a')['href']
            #세부 정보 url 열기
    #             driver = webdriver.Chrome("./chromedriver")
    #             driver.get(detail_url)
            #print("세부화면 켜기: ", detail_url)
            detail_html = requests.get(detail_url,headers=naver_headers, cookies={}).text
            body = BeautifulSoup(detail_html, 'html.parser')
    #            body = BeautifulSoup(driver.page_source, 'html.parser')
            browser = mechanicalsoup.StatefulBrowser()
            #리뷰 관련
            bodyString = str(body)
            jsonStr = '{'+bodyString[bodyString.find("ghtProductInfo"):].split('}')[0].split('{')[1]+'}'
            jsonStr = jsonStr.replace("'",'"')
            dt = json.loads(jsonStr)
            origin_id = int(dt['originalProductId'])
            review_url = f'https://series.naver.com/comments/count.json?ticket=series5&objectId={origin_id}_0&lang=ko'
            review_html = requests.get(review_url, headers = naver_headers, cookies = {}).text
            review = json.loads(review_html, encoding="utf-8")['result']['comment']
            #print(review)
            #더보기가 있으면 len = 2 없으면 len = 1
            #re.sub('\n','',url_soup.select('div._synopsis')[1].text)
            try :
                a = body.select('div._synopsis')[1].text
            except:
                a = body.select('div._synopsis')[0].text
            #desc에 쓰여있는 정규식 제거하기
            table = str.maketrans('\t\n\r\xa0\s+','       ')
            content = a.translate(table)
            #장르
            genre = body.select('li.info_lst li')[1].text
    #            driver.close()
            rx = [title,genre, detail_url, date.replace('\t',''), review, content]
            naver_novel = naver_novel.append(pd.Series(rx, index = label), ignore_index = True)
            f = open(f'naver_novel_tot.csv', 'a+', encoding = 'utf-8', newline='')
            wr = csv.writer(f)
            wr.writerow(rx)
            f.close()
        except Exception as e:
            print(e)
            if 'list index out of range' in str(e):
                continue
            else:
                time.sleep(36)
naver_novel = naver_novel[['title','genre','date','url','review','content']]

### 네이버시리즈 크롤링

In [ ]:
naver_novel = pd.DataFrame()
weekdays = ['MON','TUE','WED','THU','TRI','SAT','SUN']
label = ['title','detail_url','download','content','genre','like']
for week in weekdays:    
    list_url = f'https://novel.naver.com/webnovel/weekdayList?week={week}&genre=all&order=Read'
    naver_headers = {}
    html = requests.get(list_url, headers=naver_headers, cookies={}).text
    soup = BeautifulSoup(html, 'html.parser')
    browser = mechanicalsoup.StatefulBrowser()
    product_count = len(soup.select('ul.list_type1 li'))
    print('작품개수: ',product_count)
    for i in range(0,product_count):
        rx = []
        body = soup.select('ul.list_type1 li')[i]
        title = body.find('a')['title']
        print('제목: ', title)
        detail_url = str("https://novel.naver.com") + body.find('a')['href']
        print(detail_url)
        detail_html = requests.get(detail_url, headers=naver_headers, cookies={}).text
        detail_soup = BeautifulSoup(detail_html, 'html.parser')
        browser = mechanicalsoup.StatefulBrowser()
        download = detail_soup.select('div.section_area_info span.download')[0].get_text()
        content = detail_soup.select('div.section_area_info p.dsc')[0].get_text()
        genre = detail_soup.select('div.section_area_info span.genre')[0].get_text()
        like = detail_soup.select('div.section_area_info span.like')[0].get_text()
        rx = [title,detail_url,download,content,genre,like]
        naver_novel = naver_novel.append(pd.Series(rx, index = label), ignore_index = True)
        x = random.uniform(0.5, 2.5)
        time.sleep(x)

### 카카오페이지 top 300 크롤링

In [ ]:
label = ['title','read_count','date','genre','content','like_count','review']

# 무한 스크롤
# SCROLL_PAUSE_TIME = 2
# last_height = driver.execute_script("return document.body.scrollHeight")  
# while True:
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#     time.sleep(SCROLL_PAUSE_TIME)                                     
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);") 
#     time.sleep(SCROLL_PAUSE_TIME)

# # Calculate new scroll height and compare with last scroll height           
#     new_height = driver.execute_script("return document.body.scrollHeight")

#     if new_height == last_height:                                               
#         break

#     last_height = new_height
kakao_novel = pd.DataFrame()
page = 1

category = ['86','120','89','117','87']
for category_num in category:
    print("장르: ",category_num)
    for page in range(0,12):
        time.sleep(1.5)
        print('페이지수:',page)
        url = f'https://api2-page.kakao.com/api/v1/store/today_rank/list?page={page}&category=11&subcategory={category_num}'
        driver = webdriver.Chrome("./chromedriver")
        driver.get(url)
        body = BeautifulSoup(driver.page_source, 'html.parser').get_text()
        body = json.loads(body, encoding="utf-8")
        driver.close()
        product_count = len(body['list'])
        for i in range(0, product_count):
            rx = []
            kakao_detail = body['list'][i]
            title = kakao_detail['title']
            print('제목: ',title)
#            caption = kakao_detail['caption']
            read_count = kakao_detail['read_count']
            date = kakao_detail['last_slide_added_date']
            genre = kakao_detail['sub_category_title']
            #kakao_detail['publisher']
            content = kakao_detail['description']
            table = str.maketrans('\t\n\r\xa0\s+','       ')
            content = content.translate(table)
            like_count = kakao_detail['like_count']
            review = kakao_detail['comment_count']
            #kakao_detail['like_count']
            rx = [title, read_count, date, genre, content, like_count, review]
            kakao_novel = kakao_novel.append(pd.Series(rx, index = label), ignore_index = True)
    
    
kakao_novel = kakao_novel[['title','read_count','date','genre','content','like_count','review']]
    

### 정리

In [ ]:
naver_novel_fin = naver_novel[['title','genre','content','review_count','url']]

In [ ]:
naver_novel_fin['platform'] = '<h1>네이버</h1>'
naver_novel_fin = naver_novel_fin[['platform','title','genre','content','review_count']]

In [ ]:
kakao_novel['platform'] = '<h1>카카오</h1>'
kakao_novel_fin = kakao_novel[['platform','title','genre','content','review']]

In [ ]:
naver_novel_fin.rename(columns = {"review_count": "review"}, inplace = True)

In [ ]:
fin_novel_df = pd.concat([naver_novel_fin, kakao_novel_fin]) 

In [ ]:
fin_novel_df.to_excel('popular_novel.xlsx',header=True)

In [ ]:
naver_novel_fin['genre'].unique()

In [ ]:
kakao_novel_fin['genre'].unique()
#판타지 / 현판 / 로맨스 / 무협